In [4]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [82]:
train_dir = './small_dog_cat_dataset/train/'
test_dir = './small_dog_cat_dataset/test/'

In [83]:
os.listdir(train_dir)

['cats', 'dogs']

In [84]:
train_data = []
train_labels = []
test_data = []
test_labels = []

target_size = (64,64)


In [85]:
def read_file(path,datas,labels):
    for category in os.listdir(path):
     category_dir = os.path.join(path, category)
     for image_name in os.listdir(category_dir):
        image_path = os.path.join(category_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, target_size)
        datas.append(image)
        labels.append(category)
    return np.array(datas), np.array(labels)


In [86]:
read_file(train_dir,train_data,train_labels)
read_file(test_dir,test_data,test_labels)

(array([[[[ 64,  64,  64],
          [ 57,  57,  57],
          [ 60,  60,  60],
          ...,
          [190, 190, 190],
          [192, 192, 192],
          [191, 191, 191]],
 
         [[ 68,  68,  68],
          [ 56,  56,  56],
          [ 61,  61,  61],
          ...,
          [191, 191, 191],
          [193, 193, 193],
          [193, 193, 193]],
 
         [[ 62,  62,  62],
          [ 58,  58,  58],
          [ 65,  65,  65],
          ...,
          [192, 192, 192],
          [194, 194, 194],
          [194, 194, 194]],
 
         ...,
 
         [[157, 162, 165],
          [154, 158, 161],
          [148, 151, 154],
          ...,
          [171, 174, 179],
          [170, 173, 178],
          [170, 173, 178]],
 
         [[167, 169, 170],
          [195, 197, 198],
          [187, 189, 190],
          ...,
          [169, 173, 178],
          [168, 171, 176],
          [166, 169, 175]],
 
         [[196, 198, 199],
          [196, 198, 199],
          [200, 202, 203],
   

In [87]:
len(train_data), len(train_labels),len(test_data), len(test_labels),

(2000, 2000, 600, 600)

In [88]:

def colorspace(data):
   gray_images = []
   hsv_images = []
   for image in data:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        gray_images.append(gray)
        hsv_images.append(hsv)
   return np.array(gray_images), np.array(hsv_images)

In [92]:
train_rgb_images = train_data
train_gray_images = colorspace(train_data)[0]
train_hsv_images = colorspace(train_data)[1]
test_rgb_images = test_data
test_gray_images =colorspace(test_data)[0]
test_hsv_images = colorspace(test_data)[1]

In [93]:
len(test_hsv_images)

600

In [64]:

def HOGgrayfearture(data):
    hog_gray_features =[]
    for image in data:

        hog_features, _ = hog(image, visualize=True,
                               block_norm='L2-Hys',
                               pixels_per_cell=(16, 16),
                                cells_per_block=(2, 2))
        hog_gray_features.append(hog_features)
    return np.array(hog_gray_features)

def HOGrgbfearture(data):
    hog_gray_features =[]
    for image in data:

        hog_features, _ = hog(image, visualize=True,
                               block_norm='L2-Hys',
                               pixels_per_cell=(16, 16),
                                 cells_per_block=(2, 2),
                                 channel_axis=-1) # specify the channel axis
        hog_gray_features.append(hog_features)
    return np.array(hog_gray_features)

def HOGhsvfearture(data):
    hog_hsv_features =[]
    for image in data:
        # hue = image[:,:,0]
        hog_features, hog_image = hog(image, visualize=True,
                                       block_norm='L2-Hys',
                                         pixels_per_cell=(16, 16),
                                           cells_per_block=(2, 2),
                                           channel_axis=-1)
        hog_hsv_features.append(hog_features)
    return hog_hsv_features


In [94]:
train_rgb_feature = HOGrgbfearture(train_rgb_images)
train_gray_feature = HOGgrayfearture(train_gray_images)
train_hsv_feature = HOGhsvfearture(train_hsv_images)
test_rgb_feature = HOGrgbfearture(test_rgb_images)
test_gray_feature = HOGgrayfearture(test_gray_images)
test_hsv_feature = HOGhsvfearture(test_hsv_images)

In [50]:
len(train_rgb_feature)

2000

In [95]:
X_rgb_train, X_rgb_val, y_rgb_train, y_rgb_val, = train_test_split(train_rgb_feature,
                                                                        train_labels,
                                                                          test_size=0.1,
                                                                            random_state=50)
X_gray_train, X_gray_val, y_gray_train, y_gray_val, = train_test_split(train_gray_feature,
                                                                        train_labels,
                                                                          test_size=0.1,
                                                                            random_state=50)
X_hsv_train, X_hsv_val, y_hsv_train, y_hsv_val, = train_test_split(train_hsv_feature,
                                                                    train_labels,
                                                                      test_size=0.2,
                                                                        random_state=50)


In [96]:
clf = LogisticRegression(random_state=50)


In [97]:
rgb_clf =clf.fit(X_rgb_train, y_rgb_train)

In [98]:
gray_clf =clf.fit(X_gray_train, y_gray_train)

In [99]:
hsv_clf = clf.fit(X_hsv_train, y_hsv_train)

In [100]:
rgb_pred = rgb_clf.predict(X_rgb_val)
acc_rgb_val = accuracy_score(rgb_pred, y_rgb_val)
print("Accuracy Val:", acc_rgb_val)

test_rgb_pred = rgb_clf.predict(test_rgb_feature)
acc_rgb_test = accuracy_score(test_rgb_pred, test_labels)
print("Accuracy test:", round(acc_rgb_test, 2))

Accuracy Val: 0.72
Accuracy test: 0.72


In [116]:
gray_pred = gray_clf.predict(X_gray_val)
acc_gray_val = accuracy_score(gray_pred, y_gray_val)
print("Accuracy Val:", acc_gray_val)

test_gray_pred = gray_clf.predict(test_gray_feature)
acc_gray_test = accuracy_score(test_gray_pred, test_labels)
print("Accuracy test:", acc_gray_test)

Accuracy Val: 0.75
Accuracy test: 0.73


In [117]:
hsv_pred = gray_clf.predict(X_hsv_val)
acc_hsv_val = accuracy_score(hsv_pred, y_hsv_val)
print("Accuracy val:", acc_hsv_val)

test_hsv_pred = gray_clf.predict(test_hsv_feature)
acc_hsv_test = accuracy_score(test_hsv_pred, test_labels)
print("Accuracy test:", acc_hsv_test)

Accuracy val: 0.7275
Accuracy test: 0.6983333333333334


In [121]:
def VotingEnsembleFeature(data):
    predict = []
    rgb_images = data
    gray_images = colorspace(data)[0]
    hsv_images = colorspace(data)[1]
    rgb_feature = HOGrgbfearture(rgb_images)
    gray_feature = HOGgrayfearture(gray_images)
    hsv_feature = HOGhsvfearture(hsv_images)
    rgb_pred = rgb_clf.predict(rgb_feature)
    gray_pred = gray_clf.predict(gray_feature)
    hsv_pred = hsv_clf.predict(hsv_feature)
    for i in range(len(rgb_pred)):
        count = 0
        if rgb_pred[i] == 'dogs':
            count += 0.5
        if gray_pred[i] == 'dogs':
            count += 1
        if hsv_pred[i] == 'dogs':
            count += 0.5
        if count >= 1.5:
            predict.append('dogs')
        else:
            predict.append('cats')
    return np.array(predict)
            


In [122]:
pred = VotingEnsembleFeature(test_data)

In [123]:
acc_pred = accuracy_score(pred, test_labels)
print("Accuracy test:", acc_pred)

Accuracy test: 0.7366666666666667
